In [1]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Function to fetch and visualize data
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing=0.3)
    
    # Convert Date columns to datetime
    stock_data['Date'] = pd.to_datetime(stock_data['Date'])
    revenue_data['Date'] = pd.to_datetime(revenue_data['Date'])
    
    # Add stock price trace
    fig.add_trace(go.Scatter(x=stock_data["Date"], y=stock_data["Close"], name="Share Price"), row=1, col=1)
    
    # Add revenue trace
    fig.add_trace(go.Scatter(x=revenue_data["Date"], y=revenue_data["Revenue"], name="Revenue"), row=2, col=1)
    
    # Update axes
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)

    # Layout settings
    fig.update_layout(
        showlegend=False,
        height=900,
        title=stock,
        xaxis_rangeslider_visible=True
    )
    fig.show()

# Fetch Tesla stock data using yfinance
tesla = yf.Ticker("TSLA")

# Retrieve historical stock data
tsla_data = tesla.history(period="max")
tsla_data.reset_index(inplace=True)

# Ensure Date column is properly formatted
tsla_data['Date'] = pd.to_datetime(tsla_data['Date'])
tsla_data = tsla_data[['Date', 'Close']]  # Keeping only necessary columns

# Scrape Tesla revenue data from Macrotrends
url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Locate the correct table containing "Tesla Quarterly Revenue"
tables = soup.find_all('table')

table_index = -1  # Default value if no table is found
for index, table in enumerate(tables):
    if 'Tesla Quarterly Revenue' in table.get_text():
        table_index = index
        break

# Raise an error if the correct table isn't found
if table_index == -1:
    raise ValueError("Could not find 'Tesla Quarterly Revenue' table on the webpage.")

# Extract revenue data into DataFrame
tsla_revenue = pd.DataFrame(columns=["Date", "Revenue"])

for row in tables[table_index].find_all("tr")[1:]:  # Skip header row
    col = row.find_all("td")
    if col:
        date = col[0].text.strip()
        revenue = col[1].text.replace("$", "").replace(",", "").strip()
        
        # Append data only if revenue value is not empty
        if revenue:
            tsla_revenue = pd.concat([tsla_revenue, pd.DataFrame({'Date': [date], 'Revenue': [revenue]})], ignore_index=True)

# Convert Revenue column to numeric and handle missing values
tsla_revenue["Revenue"] = pd.to_numeric(tsla_revenue["Revenue"], errors='coerce')
tsla_revenue.dropna(inplace=True)

# Convert Date column to datetime format
tsla_revenue["Date"] = pd.to_datetime(tsla_revenue["Date"])

# Display a few rows of the revenue data
print(tsla_revenue.head())

# Generate graph with Tesla stock and revenue data
make_graph(tsla_data, tsla_revenue, 'Tesla')


ValueError: Could not find 'Tesla Quarterly Revenue' table on the webpage.